In [34]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import random
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image


In [36]:
bs = 128

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [37]:
def train(epoch, device, weight):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var, weight)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [38]:
def test(device, weight):
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var, weight).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# Task 1: Design the autoencoder structured network for MNIST

In [39]:
# YOUR CODE!!

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        
        #############################################################
    
        # YOUR CODE!!
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc3_1 = nn.Linear(h_dim2, z_dim)
        self.fc3_2 = nn.Linear(h_dim2, z_dim)

        #############################################################
        
        
        # decoder part
        
        #############################################################
    
        # YOUR CODE!!
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        #############################################################
        
        
    def encoder(self, x):
        # return mu, log_var
        
        #############################################################
    
        # YOUR CODE!!
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc3_1(h), self.fc3_2(h) # mu, log_var
        #############################################################
    
    def sampling(self, mu, log_var):
        # return z sample
        
        #############################################################
    
        # YOUR CODE!!
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        #############################################################
        
    def decoder(self, z):
        # return generated img
        
        #############################################################
    
        # YOUR CODE!!
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))
        #############################################################
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=50)
if torch.cuda.is_available():
    vae.to(device)

In [42]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3_1): Linear(in_features=256, out_features=50, bias=True)
  (fc3_2): Linear(in_features=256, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

# Task 2: Design the loss function for autoencoder with weight of KLD term

In [44]:
optimizer = optim.Adam(vae.parameters())

# #Without weight term

In [16]:

def loss_function(recon_x, x, mu, log_var, weight):
    # return reconstruction error + KL divergence losses
    
    #############################################################
    
    # YOUR CODE!!
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD
    #############################################################
    
    pass

In [17]:
from tqdm.autonotebook import tqdm

for epoch in tqdm(range(1, 51)):
    train(epoch, device, weight)
    test(device, weight)

c:\users\trojan\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 545.821350
Train Epoch: 1 [12800/60000 (21%)]	Loss: 192.805695
Train Epoch: 1 [25600/60000 (43%)]	Loss: 192.197800
Train Epoch: 1 [38400/60000 (64%)]	Loss: 164.205566
Train Epoch: 1 [51200/60000 (85%)]	Loss: 157.557327
====> Epoch: 1 Average loss: 182.2670
====> Test set loss: 145.4146
Train Epoch: 2 [0/60000 (0%)]	Loss: 142.497513
Train Epoch: 2 [12800/60000 (21%)]	Loss: 143.637848
Train Epoch: 2 [25600/60000 (43%)]	Loss: 137.245468
Train Epoch: 2 [38400/60000 (64%)]	Loss: 125.718460
Train Epoch: 2 [51200/60000 (85%)]	Loss: 123.662743
====> Epoch: 2 Average loss: 134.3210
====> Test set loss: 125.1247
Train Epoch: 3 [0/60000 (0%)]	Loss: 126.740646
Train Epoch: 3 [12800/60000 (21%)]	Loss: 120.118423
Train Epoch: 3 [25600/60000 (43%)]	Loss: 114.109962
Train Epoch: 3 [38400/60000 (64%)]	Loss: 121.798378
Train Epoch: 3 [51200/60000 (85%)]	Loss: 114.059082
====> Epoch: 3 Average loss: 120.3644
====> Test set loss: 116.5673
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 26 [25600/60000 (43%)]	Loss: 98.971985
Train Epoch: 26 [38400/60000 (64%)]	Loss: 98.550209
Train Epoch: 26 [51200/60000 (85%)]	Loss: 100.730392
====> Epoch: 26 Average loss: 99.0468
====> Test set loss: 100.1467
Train Epoch: 27 [0/60000 (0%)]	Loss: 100.841766
Train Epoch: 27 [12800/60000 (21%)]	Loss: 100.818016
Train Epoch: 27 [25600/60000 (43%)]	Loss: 101.850670
Train Epoch: 27 [38400/60000 (64%)]	Loss: 99.892288
Train Epoch: 27 [51200/60000 (85%)]	Loss: 100.751129
====> Epoch: 27 Average loss: 98.8758
====> Test set loss: 100.1053
Train Epoch: 28 [0/60000 (0%)]	Loss: 93.275352
Train Epoch: 28 [12800/60000 (21%)]	Loss: 97.266846
Train Epoch: 28 [25600/60000 (43%)]	Loss: 97.685120
Train Epoch: 28 [38400/60000 (64%)]	Loss: 99.878029
Train Epoch: 28 [51200/60000 (85%)]	Loss: 98.906952
====> Epoch: 28 Average loss: 98.7179
====> Test set loss: 99.9514
Train Epoch: 29 [0/60000 (0%)]	Loss: 97.477043
Train Epoch: 29 [12800/60000 (21%)]	Loss: 98.623230
Train Epoch: 29 [25600/6000

# #With weight term

In [45]:
def loss_function(recon_x, x, mu, log_var, weight):
    # return reconstruction error + weight * KL divergence losses
    
    #############################################################
    
    # YOUR CODE!!
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + weight*KLD
    #############################################################
    
    pass

In [46]:

# YOU HAVE TO CONTROL THE "weight" Variable!

###############################################

weight = 0.7 # change the value,

###############################################

for epoch in tqdm(range(1, 51)):
    train(epoch, device, weight)
    test(device, weight)

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.758545
Train Epoch: 1 [12800/60000 (21%)]	Loss: 193.983505
Train Epoch: 1 [25600/60000 (43%)]	Loss: 179.778244
Train Epoch: 1 [38400/60000 (64%)]	Loss: 167.685806
Train Epoch: 1 [51200/60000 (85%)]	Loss: 147.305252
====> Epoch: 1 Average loss: 177.5859
====> Test set loss: 137.9021
Train Epoch: 2 [0/60000 (0%)]	Loss: 134.590347
Train Epoch: 2 [12800/60000 (21%)]	Loss: 129.942276
Train Epoch: 2 [25600/60000 (43%)]	Loss: 132.193085
Train Epoch: 2 [38400/60000 (64%)]	Loss: 123.798737
Train Epoch: 2 [51200/60000 (85%)]	Loss: 120.639763
====> Epoch: 2 Average loss: 124.8393
====> Test set loss: 114.1375
Train Epoch: 3 [0/60000 (0%)]	Loss: 111.959023
Train Epoch: 3 [12800/60000 (21%)]	Loss: 106.714256
Train Epoch: 3 [25600/60000 (43%)]	Loss: 113.893631
Train Epoch: 3 [38400/60000 (64%)]	Loss: 112.939980
Train Epoch: 3 [51200/60000 (85%)]	Loss: 107.462067
====> Epoch: 3 Average loss: 111.1574
====> Test set loss: 106.9192
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 26 [51200/60000 (85%)]	Loss: 94.353508
====> Epoch: 26 Average loss: 91.4013
====> Test set loss: 92.7872
Train Epoch: 27 [0/60000 (0%)]	Loss: 94.463806
Train Epoch: 27 [12800/60000 (21%)]	Loss: 92.187889
Train Epoch: 27 [25600/60000 (43%)]	Loss: 93.612259
Train Epoch: 27 [38400/60000 (64%)]	Loss: 91.419830
Train Epoch: 27 [51200/60000 (85%)]	Loss: 91.161423
====> Epoch: 27 Average loss: 91.3030
====> Test set loss: 92.4349
Train Epoch: 28 [0/60000 (0%)]	Loss: 86.154785
Train Epoch: 28 [12800/60000 (21%)]	Loss: 92.118546
Train Epoch: 28 [25600/60000 (43%)]	Loss: 89.845802
Train Epoch: 28 [38400/60000 (64%)]	Loss: 93.167534
Train Epoch: 28 [51200/60000 (85%)]	Loss: 88.984200
====> Epoch: 28 Average loss: 91.1649
====> Test set loss: 92.6114
Train Epoch: 29 [0/60000 (0%)]	Loss: 90.739441
Train Epoch: 29 [12800/60000 (21%)]	Loss: 87.735016
Train Epoch: 29 [25600/60000 (43%)]	Loss: 91.387077
Train Epoch: 29 [38400/60000 (64%)]	Loss: 88.754829
Train Epoch: 29 [51200/60000 (85%)

Convergence with weight = 0.7 is faster than no weight

# Task 3

In [47]:
with torch.no_grad():
    
    #########################################################
    
    # YOUR CODE!!
    
    
    #########################################################
    
            
    z = torch.tensor(z).to(device)
    sample = vae.decoder(z.float())
    
    if not os.exists('./samples'):
        os.makedirs('./samples')
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample' + '.png')

NameError: name 'z' is not defined

In [63]:
with torch.no_grad():
    
    #########################################################
    
    # YOUR CODE!!
    z = torch.randn(64, 50).to(device)
    sample = vae.decoder(z).to(device)
    
    
    #########################################################
    
            
    #z = torch.tensor(z).to(device)
    #sample = vae.decoder(z.float())
    
    if not os.path.exists('./samples'):
        os.makedirs('./samples')
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_50D' + '.png')